In [ ]:
import ee

In [ ]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
# load study area

fc = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
roi = fc.filter(ee.Filter.eq('ADM1_NAME', 'Zuid-holland'))

## get sentinel-1 sar data

In [ ]:
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD')
asc = sentinel_1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filter(ee.Filter.eq('instrumentMode', 'IW'))
platform = asc.filter(ee.Filter.eq('platform_number', 'A'))
coll_param = platform.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).select(['VV', 'VH'])

sentinel_roi = coll_param.filterBounds(roi.geometry())

bcoff2 = sentinel_roi.filterDate('2020-03-25', '2021-01-01').first()

task_config_s1 = {
    'maxPixels': 1e13,
    'scale': 10,
    'crs': 'EPSG:4326',
    'region': roi.geometry()
    }

sarfull_task = ee.batch.Export.image(bcoff2, 'sarval_2503_vv_vh', task_config_s1)

sarfull_task.start()

print('check your gdrive in some time!')

## get landsat-8 lst data

In [ ]:
# collect 100 m lst product

landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

tile = landsat.filter(ee.Filter.eq('WRS_PATH', '198')).filter(ee.Filter.eq('WRS_ROW', '24'))
landsat_roi = landsat.filterBounds(roi.geometry())
landsatST = landsat_roi.select('ST_B10')

landST = landsatST.filterDate('2020-03-25', '2020-09-18').first()

landsat_100m = landST.resample('bilinear').reproject(crs='EPSG:4326', scale=100)

task_config_l8 = {
    'maxPixels': 1e13,
    'scale': 100,
    'crs': 'EPSG:4326',
    'region': roi.geometry()
    }

landsat_task = ee.batch.Export.image(landsat_100m, 'landsatval_2503_100', task_config_l8)

landsat_task.start()

print('check your gdrive in some time!')

## get ESAworldcover v100 data

In [ ]:
# load landcover product

data = ee.ImageCollection("ESA/WorldCover/v100")
lulc_roi = data.filterBounds(roi.geometry()).first()
lulc = lulc_roi.select('Map')

task_config_lulc = {
    'maxPixels': 1e13,
    'scale': 10,
    'crs': 'EPSG:4326',
    'region': roi.geometry()
    }

lulc_task = ee.batch.Export.image(lulc, 'esa_lulc_10', task_config_lulc)

lulc_task.start()

print('check your gdrive in some time!')

## get Sentinel-2 MSI data

In [ ]:
s2_data = ee.ImageCollection('COPERNICUS/S2_SR')

s2_roi = s2_data.filterBounds(roi.geometry())

s2_img_coll = s2_roi.filterDate('2020-03-26', '2020-03-27').select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])

s2_img_coll_mos = s2_img_coll.mosaic()

task_config_s2 = {
    'maxPixels': 1e13,
    'scale': 10,
    'crs': 'EPSG:4326',
    'region': roi.geometry()
    }

s2_task = ee.batch.Export.image(s2_img_coll_mos, 's2_2603_10', task_config_s2)

s2_task.start()

print('check your gdrive in some time!')